# Testing lightcurve plots

In [84]:
#!/usr/bin/env python3
# type: ignore
import os
import re
import sys
import time
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import requests

# Define the output directory
output_dir = r'C:\Users\jhzhe\Documents\output_test'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)


BASEURL = "https://fallingstar-data.com/forcedphot"
# BASEURL = "http://127.0.0.1:8000"

if os.environ.get("ATLASFORCED_SECRET_KEY"):
    token = os.environ.get("ATLASFORCED_SECRET_KEY")
    print("Using stored token")
else:
    data = {"username": "JinZheng", "password": "Swagrider2015"}

    resp = requests.post(url=f"{BASEURL}/api-token-auth/", data=data)

    if resp.status_code == 200:
        token = resp.json()["token"]
        print(f"Your token is {token}")
        print("Store this by running/adding to your .zshrc file:")
        print(f'export ATLASFORCED_SECRET_KEY="{token}"')
    else:
        print(f"ERROR {resp.status_code}")
        print(resp.text)
        sys.exit()


headers = {"Authorization": f"Token {token}", "Accept": "application/json"}

Your token is 938191f3499bfce3ac5b3566e298b2c9106d84d3
Store this by running/adding to your .zshrc file:
export ATLASFORCED_SECRET_KEY="938191f3499bfce3ac5b3566e298b2c9106d84d3"


In [85]:
import random

RA_min = 133
RA_max = 135
dec_min = 13
dec_max = 15

RA_list = []
dec_list = []

for i in range(3):
    RA_list.append(round(random.uniform(RA_min, RA_max),2))
    dec_list.append(round(random.uniform(dec_min, dec_max),2))



In [86]:

def fetch_data(ra, dec, mjd_min=58582.8, index=1):
    task_url = None
    while not task_url:
        with requests.Session() as s:
            # alternative to token auth
            # s.auth = ('USERNAME', 'PASSWORD')
            resp = s.post(f"{BASEURL}/queue/", headers=headers, data={"ra": ra, "dec": dec, "mjd_min": mjd_min})

            if resp.status_code == 201:  # successfully queued
                task_url = resp.json()["url"]
                print(f"The task URL is {task_url}")
            elif resp.status_code == 429:  # throttled
                message = resp.json()["detail"]
                print(f"{resp.status_code} {message}")
                t_sec = re.findall(r"available in (\d+) seconds", message)
                t_min = re.findall(r"available in (\d+) minutes", message)
                if t_sec:
                    waittime = int(t_sec[0])
                elif t_min:
                    waittime = int(t_min[0]) * 60
                else:
                    waittime = 10
                print(f"Waiting {waittime} seconds")
                time.sleep(waittime)
            else:
                print(f"ERROR {resp.status_code}")
                print(resp.text)
                sys.exit()


    result_url = None
    taskstarted_printed = False
    while not result_url:
        with requests.Session() as s:
            resp = s.get(task_url, headers=headers)

            if resp.status_code == 200:  # HTTP OK
                if resp.json()["finishtimestamp"]:
                    result_url = resp.json()["result_url"]
                    print(f"Task is complete with results available at {result_url}")
                elif resp.json()["starttimestamp"]:
                    if not taskstarted_printed:
                        print(f"Task is running (started at {resp.json()['starttimestamp']})")
                        taskstarted_printed = True
                    time.sleep(2)
                else:
                    print(f"Waiting for job to start (queued at {resp.json()['timestamp']})")
                    time.sleep(4)
            else:
                print(f"ERROR {resp.status_code}")
                print(resp.text)
                sys.exit()

    with requests.Session() as s:
        textdata = s.get(result_url, headers=headers).text

        # if we'll be making a lot of requests, keep the web queue from being
        # cluttered (and reduce server storage usage) by sending a delete operation
        # s.delete(task_url, headers=headers).json()

    dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)
    file_path_image = os.path.join(output_dir, f'plot_data{index}.png')
    file_path_data = os.path.join(output_dir, f'data{index}.csv')
    
    dfresult.to_csv(file_path_data)

    fig = plt.figure()
    plt.plot(dfresult['MJD'], dfresult['uJy'])
    plt.savefig(file_path_image)
    plt.close(fig)


In [87]:
for i in range(len(dec_list)):
    fetch_data(RA_list[i], dec_list[i], 60581.7, i)


    

The task URL is https://fallingstar-data.com/forcedphot/queue/1800216/
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1800216.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_23300\2815477301.py:61: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


The task URL is https://fallingstar-data.com/forcedphot/queue/1800217/
Task is running (started at 2024-10-28T20:02:18Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1800217.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_23300\2815477301.py:61: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)


The task URL is https://fallingstar-data.com/forcedphot/queue/1800220/
Task is running (started at 2024-10-28T20:02:37Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1800220.txt


C:\Users\jhzhe\AppData\Local\Temp\ipykernel_23300\2815477301.py:61: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)
